In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
df = pd.read_csv('BD/P2_G5_films.csv.gz', compression = 'gzip', na_values = ['\\N'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_1042/2534545640.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('BD/P2_G5_films.csv.gz', compression = 'gzip', na_values = ['\\N'])


In [57]:
''' 
Il faudrait ajouter ce code plus tôt dans les merges des autres tables afin de collecter les données revenues et budget associées.
On en reparle.
'''

df_str = df.select_dtypes(exclude = 'number')
df_str_drop_dup = df_str.drop_duplicates(subset = ['film_id'], keep = 'first')

df_num = pd.concat([df['film_id'], df.select_dtypes(include = 'number')], axis = 1)
df_num = df_num.drop(['revenue', 'budget'], axis = 1)

df_revenue_budget = df[['film_id', 'revenue', 'budget']]
df_revenue_budget_gb = df_revenue_budget.groupby('film_id').max()

df_num_gb = df_num.groupby('film_id').max()

df_num2 = pd.merge(df_num_gb,
                   df_revenue_budget_gb,
                   how = 'outer',
                   on = 'film_id')
df_test = pd.merge(df_str_drop_dup,
                   df_num2,
                   on = 'film_id',
                   how = 'left')

In [64]:
df_test[df_test['title'].str.contains('Avatar') == True].transpose()

,25591,29155,55499,56256
adult,False,False,False,False
backdrop_path,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,/8rpDcsfLJypbO6vREc0547VKqEv.jpg,NaN,NaN
homepage,https://www.avatar.com/movies/avatar,https://www.avatar.com/movies/avatar-the-way-o...,https://www.avatar.com/movies,https://www.avatar.com/movies
overview,"In the 22nd century, a paraplegic Marine is di...",Set more than a decade after the events of the...,NaN,NaN
poster_path,/jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,/qls9tbBTTDDEXuqm9kLB16Y7xhE.jpg,/rtmmvqkIC5zDMEd638Es2woxbz8.jpg
production_countries,"['US', 'GB']",['US'],['US'],['US']
status,Released,Released,In Production,In Production
tagline,Enter the world of Pandora.,Return to Pandora.,NaN,NaN
video,False,False,False,False
production_companies_name,"['Dune Entertainment', 'Lightstorm Entertainme...","['20th Century Studios', 'Lightstorm Entertain...","['Lightstorm Entertainment', '20th Century Stu...","['Lightstorm Entertainment', '20th Century Stu..."


In [58]:
len(df), len(df_test), len(df_str_drop_dup), len(df_num_gb), len(df_num), len(df_revenue_budget), len(df_num2)

(338718, 56336, 56336, 56336, 338718, 338718, 56336)

In [69]:
'''
ATTENTION ERREUR AVEC CETTE RECHERCHE
'''

df_tmbd_full = pd.read_csv('BD/tmdb_full.csv')
df_tmbd_full[df_tmbd_full['title'].str.contains('Avatar 3') == True].transpose()

,203424
adult,False
backdrop_path,NaN
budget,250000000
genres,"['Action', 'Science Fiction', 'Adventure']"
homepage,https://www.avatar.com/movies
id,83533
imdb_id,tt1757678
original_language,en
original_title,Avatar 3
overview,The third entry in the Avatar franchise.


In [50]:
colonnes = df_test.columns
donnees = pd.DataFrame(columns = colonnes)

for col in colonnes:
    donnees.loc[0, col] = round((len(df_test[df_test[col] != 0]) / len(df_test))*100, 2)

donnees.transpose()

,0
adult,0.28
backdrop_path,100.0
homepage,100.0
overview,100.0
poster_path,100.0
production_countries,100.0
status,100.0
tagline,100.0
video,11.93
production_companies_name,100.0


In [108]:
colonnes = df.columns
donnees = pd.DataFrame(columns = colonnes)

for col in colonnes:
    donnees.loc[0, col] = round((len(df[df[col] != 0]) / len(df))*100, 2)

donnees.transpose()

,0
adult,1.8
backdrop_path,100.0
budget,5.53
homepage,100.0
id,100.0
overview,100.0
popularity,21.99
poster_path,100.0
production_countries,100.0
revenue,5.58


In [65]:
(len(df[df['revenue'] != 0]) / len(df))*100

5.57858047868119

In [107]:
len(df)

1508269

In [89]:
df_sorted = df.sort_values(by = 'final_language', ascending = False)

In [91]:
df_sorted = df_sorted.drop_duplicates(subset = 'film_id', keep='first')
len(df_sorted)

541763

In [98]:
colonnes = df_sorted.columns
donnees = pd.DataFrame(columns = colonnes)

for col in colonnes:
    donnees.loc[0, col] = round((len(df_sorted[df_sorted[col] != 0]) / len(df_sorted))*100, 2)

donnees.transpose()

,0
adult,1.64
backdrop_path,100.0
budget,1.92
homepage,100.0
id,100.0
overview,100.0
popularity,9.2
poster_path,100.0
production_countries,100.0
revenue,1.83


In [101]:
df_revenue = df[['film_id', 'revenue', 'budget']]
df_revenue = df_revenue.sort_values(by = 'revenue', ascending = False)
df_revenue = df_revenue.drop_duplicates(subset = 'film_id', keep='first')

In [105]:
df_final = pd.merge(df_sorted,
                    df_revenue,
                    how = 'left',
                    on = 'film_id')

In [106]:
colonnes = df_final.columns
donnees = pd.DataFrame(columns = colonnes)

for col in colonnes:
    donnees.loc[0, col] = round((len(df_final[df_final[col] != 0]) / len(df_final))*100, 2)

donnees.transpose()

,0
adult,1.64
backdrop_path,100.0
budget_x,1.92
homepage,100.0
id,100.0
overview,100.0
popularity,9.2
poster_path,100.0
production_countries,100.0
revenue_x,1.83
